In [ ]:

import os
import pandas as pd
import nltk
import re 
import seaborn as sns 
import matplotlib.pyplot as plt
import streamlit

# Set the path to the directory where your CSV files are stored
path = 'D:/HUMAN_RESOURCE_MANAGMENT'  # replace with your CSV files' directory path
all_files = [os.path.join(path, f) for f in os.listdir(path) if f.endswith('.csv')]

# List to hold dataframes
df_list = []

# Loop through all CSV files and append to the list
for filename in all_files:
    
    df = pd.read_csv(filename, index_col=None, header=0, encoding='ISO-8859-1')
    df_list.append(df)

# Concatenate all dataframes into a single dataframe
data= pd.concat(df_list, axis=0, ignore_index=True)

# Now you have a new dataframe 'combined_df' with all your CSV data

data
# This will display the DataFrame with some default styling


In [ ]:
data.rename(columns={
    'NIC Name': 'NIC_NAME',
    "India/States": "state_ut",
    'State Code': 'State_Code',
    'District Code': 'District_Code'
}, inplace=True)

In [ ]:
data["NIC_NAME"]=="Incomplete description/ Wrongly Classifed"

In [ ]:
data=data[~data['NIC_NAME'].isin(["Blank","Incomplete description/ Wrongly Classifed","incomplete","BLANK","Blank etc."])]
data

In [ ]:
data['state_ut']

total words state wise

In [ ]:
h=data[data["NIC_NAME"].str.lower()=="total"]
df = h[h['state_ut'].str.startswith('STATE')]


state totals dataframe

In [ ]:
df

datasets exported here

In [ ]:
data.reset_index(drop=True)

In [ ]:
data

In [ ]:
pd.DataFrame(data.state_ut.unique())

In [ ]:
data.state_ut.value_counts()

In [ ]:
data

In [ ]:
data["state_ut"]=data["state_ut"].str.lower()
data["NIC_NAME"]=data["NIC_NAME"].str.lower()

state only data

In [ ]:
state_mask = data['state_ut'].str.startswith('state')
state_df = data[state_mask]
state_df
district= data['state_ut'].str.startswith('district')
dist_df=data[district]
dist_df


In [38]:

hg=dist_df[dist_df["NIC_NAME"]=="total"]
hg.drop_duplicates(subset=['District_Code', 'state_ut'], keep='last')
hg.to_csv('D:/total_NIC_dist.csv', index=False)


hg=state_df[state_df["NIC_NAME"]=="total"]
hg.drop_duplicates(subset=['State_Code', 'state_ut'], keep='last')
hg.to_csv('D:/total_NIC_state.csv', index=False)

In [ ]:
stdup=state_df.state_ut.value_counts()
stdup

In [ ]:
distdup=dist_df.state_ut.value_counts().duplicated
distdup

removing duplicates

In [ ]:
new= data.drop_duplicates(subset=['State_Code', 'District_Code', 'state_ut', 'Division',
        'Main Workers - Total -  Persons',
       'Main Workers - Total - Males', 'Main Workers - Total - Females',
       'Main Workers - Rural -  Persons', 'Main Workers - Rural - Males',
       'Main Workers - Rural - Females', 'Main Workers - Urban -  Persons',
       'Main Workers - Urban - Males', 'Main Workers - Urban - Females',
       'Marginal Workers - Total -  Persons',
       'Marginal Workers - Total - Males',
       'Marginal Workers - Total - Females',
       'Marginal Workers - Rural -  Persons',
       'Marginal Workers - Rural - Males',
       'Marginal Workers - Rural - Females',
       'Marginal Workers - Urban -  Persons',
       'Marginal Workers - Urban - Males',
       'Marginal Workers - Urban - Females'], keep='last')

In [ ]:
new.columns

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

# Tokenize the text in the 'text_column'
new['tokenized_NIC'] = new['NIC_NAME'].apply(lambda text: nltk.word_tokenize(text))
def stemmed(tokens):
    post=PorterStemmer()
    txt=" ".join([post.stem(word) for word in tokens])
    return txt

def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'))
    return [word for word in tokens if word.lower() not in stop_words]

def lemmatize(tokens):
    lemmatizer = nltk.WordNetLemmatizer()
    text=" ".join([lemmatizer.lemmatize(word,pos='v') for word in tokens])
    return text

new['removed_stops'] = new['tokenized_NIC'].apply(remove_stopwords)
new['lemmatized'] = new['removed_stops'].apply(lemmatize)



In [ ]:
new.reset_index(drop=True)

In [ ]:
new.describe()

In [ ]:
new.size

statewise entries

In [ ]:
entries=pd.DataFrame(new.State_Code.value_counts())
#top=entries.max()
#low=entries.min()


In [ ]:
d=pd.DataFrame(new.District_Code.value_counts())
d.sort_index

In [ ]:

nic=pd.DataFrame(new.NIC_NAME.value_counts())
hj=nic.sort_index()
hj[25:50]

In [ ]:
hj[200:225]

In [ ]:
new.isnull().sum()

In [ ]:
new.District_Code.isna()


In [ ]:
import seaborn as sns 
ax=sns.displot(new["State_Code"])
ax.tick_params(axis='x',rotation=90)

In [ ]:
data["District_Code"].unique()

In [ ]:
'''from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

# Our sentences we like to encode
sentences = [new['lemmatized']
]

# Sentences are encoded by calling model.encode()
sentence_embeddings = model.encode(sentences)

sentence_embeddings'''

In [ ]:
new

In [ ]:
new.head()

total word in nic


In [ ]:
totalvals=new[new['NIC_NAME']=="total"]
tl=totalvals.District_Code.to_list()

In [ ]:

new['lemmatized'].count()
new=new[~new['lemmatized'].isin(["total"])]
new.shape

In [ ]:
unique_values = new['lemmatized'].unique().tolist()
len(unique_values)

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans

# Load a pre-trained Sentence-Transformer model (e.g., 'paraphrase-MiniLM-L6-v2')
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

unique_values
embeddings = model.encode(unique_values)
from sklearn.cluster import KMeans

num_clusters = 4 # Example number of clusters
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
clusters = kmeans.fit_predict(embeddings)


df5 = pd.DataFrame({'lemmatized': unique_values})




In [ ]:

df5['embeddings'] = df5['lemmatized'].apply(lambda x: model.encode(x))

In [ ]:
new

In [ ]:
new11=new.join(df5.set_index('lemmatized'), on='lemmatized', validate='m:1')



In [ ]:
new11

In [ ]:

# Apply K-means clustering
num_clusters = 4 # Specify the number of clusters
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
new11['cluster'] = kmeans.fit_predict(list(new11['embeddings']))

# Print the clustered sentences
'''for cluster_id in range(num_clusters):
    cluster_sentences = new1[new1['cluster'] == cluster_id]['lemmatized']
    print(f"Cluster {cluster_id + 1} ({len(cluster_sentences)} sentences):")
    print(cluster_sentences.tolist())
    

# Optional: You can explore the cluster centroids (cluster centers)
cluster_centers = kmeans.cluster_centers_
print("Cluster Centers (Embeddings):")
print(cluster_centers)'''

# Optional: Assign meaningful labels to clusters based on content
# For example, you can manually inspect the sentences in each cluster
# and label them accordingly.

# Note: Adjust the model, data, and parameters according to your specific use case.


In [ ]:
new11

In [ ]:
new11.describe()

In [ ]:
newdf1 = new11[new11['cluster'] == 0]
newdf1.to_csv('D:/clusteris0.csv', index=False)  # Specify the desired file name
newdf2 = new11[new11['cluster'] == 1]
newdf2.to_csv('D:/clusteris1.csv', index=False)
newdf3 = new11[new11['cluster'] == 2]
newdf3.to_csv('D:/clusteris2.csv', index=False)
newdf4 = new11[new11['cluster'] == 3]
newdf4.to_csv('D:/clusteris4.csv', index=False)
